In [1]:
import requests
import re
import json
import pandas as pd
from datetime import datetime, timedelta
import ast
import warnings
warnings.simplefilter("ignore", UserWarning)

In [2]:

URL_BASE = "https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute"

ativo = 'TAEE11'

params = {
    "rpcids": "AiCwsd",
    "source-path": "/finance/quote/BBAS3:BVMF",
    "f.sid": "-1601762218538142130",
    "bl": "boq_finance-ui_20251010.01_p1",
    "hl": "pt",
    "soc-app": "162",
    "soc-platform": "1",
    "soc-device": "1",
    "_reqid": "39577530",
    "rt": "c",
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Content-Type": "application/x-www-form-urlencoded;charset=utf-8",
    "Referer": "https://www.google.com/finance/quote/BBAS3:BVMF"
}

payload = (
    f'f.req=[[["AiCwsd","[[[null,[\\"{ativo}\\",\\"BVMF\\"]]],1,null,null,null,null,null,0]",null,"generic"]]]'
    '&at=ANXjBT-5oBApzzVabFGFEcU480EA:1760488320351'
)


response = requests.post(URL_BASE, params=params, headers=headers, data=payload)

print(response.status_code)

bronze = response.text

bronze

200


')]}\'\n\n35147\n[["wrb.fr","AiCwsd","[[[[\\"TAEE11\\",\\"BVMF\\"],\\"/g/11dx9hkgyy\\",\\"BRL\\",[[[1,[2025,10,28,10,null,null,null,[-10800]],[2025,10,28,16,56,null,null,[-10800]]],[[[2025,10,28,10,2,null,null,[-10800]],[37.52,0.20000000000000284,0.00535905680600222,2,2,4],522000],[[2025,10,28,10,3,null,null,[-10800]],[37.64,0.3200000000000003,0.008574490889603437,2,2,4],9200],[[2025,10,28,10,4,null,null,[-10800]],[37.56,0.240000000000002,0.006430868167202625,2,2,4],4300],[[2025,10,28,10,5,null,null,[-10800]],[37.62,0.29999999999999716,0.008038585209003139,2,2,4],2500],[[2025,10,28,10,6,null,null,[-10800]],[37.63,0.3100000000000023,0.008306538049303383,2,2,4],5200],[[2025,10,28,10,7,null,null,[-10800]],[37.61,0.28999999999999915,0.007770632368703085,2,2,4],13800],[[2025,10,28,10,8,null,null,[-10800]],[37.66,0.3399999999999963,0.009110396570203546,2,2,4],2100],[[2025,10,28,10,9,null,null,[-10800]],[37.65,0.3299999999999983,0.008842443729903492,2,2,4],2600],[[2025,10,28,10,10,null,null,[

In [3]:
pattern_primeira_linha = r'\[\[2025,\d+,\d+,\d+.*?\]\]\](?=,null,-10800)'

match = re.search(pattern_primeira_linha, bronze, re.DOTALL)

bronze_json = match.group(0).replace('[[', '[').replace(']]', ']')

bronze_json


'[2025,10,28,10,2,null,null,[-10800],[37.52,0.20000000000000284,0.00535905680600222,2,2,4],522000],[2025,10,28,10,3,null,null,[-10800],[37.64,0.3200000000000003,0.008574490889603437,2,2,4],9200],[2025,10,28,10,4,null,null,[-10800],[37.56,0.240000000000002,0.006430868167202625,2,2,4],4300],[2025,10,28,10,5,null,null,[-10800],[37.62,0.29999999999999716,0.008038585209003139,2,2,4],2500],[2025,10,28,10,6,null,null,[-10800],[37.63,0.3100000000000023,0.008306538049303383,2,2,4],5200],[2025,10,28,10,7,null,null,[-10800],[37.61,0.28999999999999915,0.007770632368703085,2,2,4],13800],[2025,10,28,10,8,null,null,[-10800],[37.66,0.3399999999999963,0.009110396570203546,2,2,4],2100],[2025,10,28,10,9,null,null,[-10800],[37.65,0.3299999999999983,0.008842443729903492,2,2,4],2600],[2025,10,28,10,10,null,null,[-10800],[37.51,0.18999999999999773,0.0050911039657019755,2,2,4],9000],[2025,10,28,10,11,null,null,[-10800],[37.52,0.20000000000000284,0.00535905680600222,2,2,4],7100],[2025,10,28,10,12,null,null,[-1

In [4]:
pat = '\[\[\[\d{4},\d{1,2},\d{1,2},\d{1,2},\d{1,2},null,null,\[-?\d+\]\],\[[0-9.,\-eE]+(?:,[0-9.,\-eE]+)*\],\d+\],'

matches = re.findall(pat, bronze, re.DOTALL)

df = pd.DataFrame(matches)

df['raw'] = df[0]

matches

<>:1: SyntaxWarning: invalid escape sequence '\['
<>:1: SyntaxWarning: invalid escape sequence '\['
C:\Users\Maicon Ferreira\AppData\Local\Temp\ipykernel_2924\2500998122.py:1: SyntaxWarning: invalid escape sequence '\['
  pat = '\[\[\[\d{4},\d{1,2},\d{1,2},\d{1,2},\d{1,2},null,null,\[-?\d+\]\],\[[0-9.,\-eE]+(?:,[0-9.,\-eE]+)*\],\d+\],'


['[[[2025,10,28,10,2,null,null,[-10800]],[37.52,0.20000000000000284,0.00535905680600222,2,2,4],522000],']

In [5]:
df = df.drop(columns=(0))

cols = df['raw'].str.split(r',\[', expand=True)

cols.columns = [f'parte_{i+1}' for i in range(cols.shape[1])]


In [6]:

silver = (
    pd
    .concat([df, cols], axis=1)
    .drop(columns=['raw', 'parte_2'], axis=1)
    )

silver['parte_1'] = silver['parte_1'].str.replace('[', '').replace(']', '').replace('null', '')

silver['price'] = silver['parte_3'].str.extract('([^,]+)')


silver[['year', 'month', 'day', 'hour', 'minutes']] = silver['parte_1'].str.split(',', expand=True).iloc[:, :5]

silver['datetime'] = pd.to_datetime({
    'year': silver['year'],
    'month': silver['month'],
    'day': silver['day'],
    'hour': silver['hour'],
    'minute': silver['minutes']
})

silver = silver.drop(columns=['parte_3', 'parte_1'])

silver['ativo'] = ativo

silver

,price,year,month,day,hour,minutes,datetime,ativo
0,37.52,2025,10,28,10,2,2025-10-28 10:02:00,TAEE11
